In [1]:
import os
from dotenv import load_dotenv
from llama_index.core import Settings
load_dotenv()

True

In [2]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

# for Azure OpenAI model
api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
gpt_api_version = os.getenv('AZURE_GPT_API_VERSION')
embedding_api_version = os.getenv('AZURE_EMBEDDING_API_VERSION')

llm = AzureOpenAI(
    model="gpt-4o",
    deployment_name="gpt-4o",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=gpt_api_version,
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name="text-embedding-3-small",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=embedding_api_version,
)   

Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
# Load PDF documents
# Change 'data' to the directory containing your PDFs
documents = SimpleDirectoryReader(input_dir="../data/stage4/").load_data()
index = VectorStoreIndex.from_documents(documents)
retriever = index.as_retriever(similarity_top_k=10)

In [10]:
from llama_index.core.workflow import Context
    
async def search_documents(query: str) -> str:
    """Search the PDF documents for information on a given topic."""
    retrieval_results = retriever.retrieve(query)
    contexts = [node.text for node in retrieval_results]
    return "\n\n".join([f"Document chunk {i+1}:\n{context}" for i, context in enumerate(contexts)])

async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes based on research done on guidelines. Your input should be notes with a title to save the notes under."""
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."

async def write_email(ctx: Context, email: str) -> str:
    """Useful for writing and updating a report. Your input should be a markdown formatted report section."""
    current_state = await ctx.get("state")
    current_state["customer_email"] = email
    await ctx.set("state", current_state)
    print (f"\n Email content: {email}")
    return "email sent."

async def move_to_next_stage() -> str:
    """Useful for writing and updating a report. Your input should be a markdown formatted report section."""
    print("All good -> moving to data duplication check stage")
    return "Moving to next stage."

In [11]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the Underwriter guideline PDF to analyse if the broker's quote sumission meets all guideline requirements.",
    system_prompt=(
        """
        You are an insurance underwriter research assistant. Your job is to analyze insurance quote submissions and check if they meet the company's underwriting guidelines by researching the guideline documents.

        When you receive a quote submission:
        1. Identify the key elements (location, type of property, coverage requested, etc.)
        2. Use the search_documents tool to retrieve relevant guideline sections for each key element
        3. Compare the submission against the guidelines meticulously
        4. Record detailed notes using the record_notes tool on whether the submission meets or violates guidelines
        5. Look for guidelines related to hazards, vulnerabilites, risks, limits, and deductibles.

        Your notes should be comprehensive and clearly indicate if there are compliance issues that would prevent policy issuance.

        When you've completed your analysis:
        - Summarize your findings
        - Clearly state if the submission meets guidelines or has issues
        - Hand off to the EmailAgent with your detailed findings

        Be thorough and precise - your research is critical for proper compliance and risk assessment.
        Always hand off to the EmailAgent for further action.
    """
    ),
    llm=llm,
    tools=[search_documents, record_notes],
    can_handoff_to=["EmailAgent"],
)


email_agent = FunctionAgent(
    name="EmailAgent",
    description="Useful for drafting and sending emails to brokers regarding the guideline compliance of their quote submissions based on the research findings.",
    system_prompt=(
        """
        You are an insurance company email communications specialist. Your responsibility is to draft clear, professional emails to insurance brokers based on research findings about their quote submissions.

        When you receive a handoff from the ResearchAgent:
        1. Carefully review all research notes and findings
        2. Determine if the submission meets all underwriting guidelines

        If the submission DOES NOT meet guidelines:
        - Draft a polite but direct email to the broker using the write_email tool
        - Clearly explain which guidelines were not met
        - Provide specific details from the research
        - Include instructions on what needs to be addressed for resubmission
        - Maintain a professional tone throughout
        - Format the email with proper salutation, body, and closing

        If the submission MEETS ALL guidelines:
        - Use the move_to_next_stage tool to advance the workflow
        - No email needs to be sent in this case

        Your emails should be clear, concise, and helpful to brokers while maintaining the company's standards and compliance requirements.
       """
    ),
    llm=llm,
    tools=[write_email, move_to_next_stage]
)

In [12]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, email_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "customer_email": "not drafted yet."
    },
)

In [13]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = agent_workflow.run(
    user_msg=(
        """
        Please triage the following property insurance quote submission from a broker:
        {
        "insurance_broker": "Prime Insurance Brokers",
        "date": "24 April 2025",
        "insurance_company": "Al Ameen Insurance",
        "address": "Office 801, Saffar Tower, Valiasr Street, Tehran, Iran",
        "recipient": "Mr. David Thompson",
        "subject": "Request for Property Insurance Quote for Parsian Evin Hotel Ltd.",
        "client": "Parsian Evin Hotel Ltd.",
        "property_information": {
        "location": "No. 45, Evin Street, Tehran, Iran",
        "type": "Hotel",
        "construction": "Modern design, reinforced concrete and steel, built in 2010, no recent renovations",
        "surface_area": "11,500 m²",
        "occupancy": "150-room hotel, luxury restaurant, and conference facilities"
        },
        "coverage_requirements": {
        "desired_coverage_amount": "IRR 800,000,000,000",
        "coverage_type": ["Fire", "theft", "guest property"],
        "deductibles": "IRR 500,000,000 per incident",
        "additional_coverage": [
            "Business interruption",
            "loss of revenue due to closure",
            "third-party liability"
        ]
        },
        "risk_assessment": {
        "fire_hazards": [
            "Fire alarm and sprinkler system in all rooms",
            "fire exits clearly marked"
        ],
        "natural_disasters": [
            "Low flood risk",
            "not located in an earthquake-prone area",
            "occasional sandstorms"
        ],
        "security_measures": [
            "CCTV surveillance",
            "24/7 security personnel",
            "secure entry systems"
        ]
        },
        "financial_information": {
        "property_value": "IRR 1,000,000,000,000",
        "business_revenue": "IRR 300,000,000,000 annually"
        },
        "contact_person": {
        "name": "Oliver Green",
        "email": "oliver.green@primeinsurance.com",
        "phone": "+971 4 234 5678"
        }
    }
        Please analyze this submission and determine if it meets all compliance check.
        """
    )
)

current_agent = None
current_tool_calls = ""

try:
    async for event in handler.stream_events():
        if (
            hasattr(event, "current_agent_name")
            and event.current_agent_name != current_agent
        ):
            current_agent = event.current_agent_name
            print(f"\n{'='*50}")
            print(f" 🤖 Agent: {current_agent}")
            print(f"{'='*50}\n")

        if isinstance(event, AgentStream):
            if event.delta:
                print(event.delta, end="", flush=True)
        elif isinstance(event, AgentInput):
            print("\n 📥 Input:", event.input)

        elif isinstance(event, AgentOutput):
            if event.response.content:
                print("\n 📤 Output:", event.response.content)
            if event.tool_calls:
                print(
                    "\n 🛠️  Planning to use tools:",
                    [call.tool_name for call in event.tool_calls],
                )
        elif isinstance(event, ToolCallResult):
            print(f" 🔧 Tool Result ({event.tool_name}):")
            print(f"  Arguments: {event.tool_kwargs}")
            print(f"  Output: {event.tool_output}")
        elif isinstance(event, ToolCall):
            print(f" 🔨 Calling Tool: {event.tool_name}")
            print(f"  With arguments: {event.tool_kwargs}")

except Exception as e:
    print(f"An error occurred: {e}")

response = await handler


 🤖 Agent: ResearchAgent


 📥 Input: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="\n        You are an insurance underwriter research assistant. Your job is to analyze insurance quote submissions and check if they meet the company's underwriting guidelines by researching the guideline documents.\n\n        When you receive a quote submission:\n        1. Identify the key elements (location, type of property, coverage requested, etc.)\n        2. Use the search_documents tool to retrieve relevant guideline sections for each key element\n        3. Compare the submission against the guidelines meticulously\n        4. Record detailed notes using the record_notes tool on whether the submission meets or violates guidelines\n        5. Look for guidelines related to hazards, vulnerabilites, risks, limits, and deductibles.\n\n        Your notes should be comprehensive and clearly indicate if there are compliance issues tha

********************************************************************************************************************************************************************************************
# Email
 Email content: Subject: Review of Property Insurance Quote Submission for Parsian Evin Hotel Ltd.

Dear Mr. Green,

Thank you for submitting the property insurance quote request for Parsian Evin Hotel Ltd. We have reviewed the submission against our underwriting guidelines and would like to provide feedback to ensure compliance and facilitate the process.

**Findings from the Review:**

1. **Coverage Limits:**
   - The desired coverage amount of IRR 800,000,000,000 (~$19 million USD) exceeds our property coverage limit of $10 million per location. Adjustments to the requested coverage amount are necessary to align with our guidelines.

2. **Liability Limits:**
   - The requested third-party liability coverage may exceed our maximum limit of $6 million per occurrence. Please confirm the required liability coverage amount or consider adjusting it to meet our guidelines.

3. **Guest Property Coverage:**
   - Liability for guest property is capped at $25,000 aggregate. If higher coverage is required, please provide additional details for further review.

4. **Additional Coverage:**
   - Business interruption and loss of revenue coverage are requested. Our guidelines allow loss of income coverage for up to 360 days. Please confirm if this meets your client’s requirements.

**Next Steps:**
- Kindly review the above points and provide revised coverage amounts or additional information for further evaluation.
- If you have any questions or need assistance, please do not hesitate to contact us.

We appreciate your cooperation and look forward to assisting you further.

Best regards,

[Your Name]  
Underwriting Team  
Al Ameen Insurance  